## Create a notebook to perform Runge-Kutta integration for multiple coupled variables

In [ ]:
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt

## Define our coupled derivatives to integrate 

In [ ]:
def dydx(x,y):
    #set the derivatives 
    #our equation is d^2y/dx^2 = y
    #so we can wirte
    #dy/dx= z
    #dz/dx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #declare an array 
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1]
    y_derive[1] = -1*y[0]
    
    return y_derivs

## Define the 4th order Rk method 

In [ ]:
def rk4_mv_core(dydx,xi,yi,h):
    
    #declare k arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define xat 1/2 step
    x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    x_ipo = xi+h
    
    #declare a temp y array 
    y_temp = np.zeros(nv)
    
    #get k1 values 
    y_derivs = dxdy(xi,yi)
    k1 = h*yderivs[:]
    
    #get k2 values 
    y_temp[:] = yi[:]+0.5*k1[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k2[:] = h*y_derives[:] 
    
    #get k3 values 
    y_temp[:] = yi[:]+0.5*k2[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k3[:] = h*y_derives[:] 
    
    #get k4 values
    y_temp[:] = yi[:]+0.5*k3[:]
    y_derivs = dydx(x_ipoh, y_temp)
    k4[:] = h*y_derives[:] 
    
    #advance y by a step h 
    yipo = yi + (k1+2*k2 + 2*k3 + k4)/6
    
    return yipo

In [ ]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale 
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iteration 
    imax = 10000
    
    #set an iteration variable 
    i = 0
    
    #create an error 
    Delta = np.full(nv,2*tol)
    
    #remember the step 
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        #estimate our error by taking one step of size h
        #vs two steps of size h/2
        y_2 = rk4_mv_core(dydx,x_i, y_i,nv,h_step)
        y_1 = rk4_mv_core(dydx,x_i,y_i, nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        #compute an error 
        Delta = np.fabs(y_2-y_11)
        
        #if the error is too large, take a smaller step 
        if(Delta.max()/tol>1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>imax):
            print('too many iterations in rk4_mv_ad()')
            raise StopIteration('ending after i = ',i)
            
        #iterate 
        i += 1 
        
    #next time, try to take a bigger step 
    h_new = np.fmin(h_step * (delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the stp we actually took 
    return y_2, h_new, h_step

## Define a wrapper for RK4

In [ ]:
def rk4_mv(dfdx,a,b,y_a,tol):
    
    #dfdx is the derivative wrt x
    #a is the lower bound 
    #b is the upper bound 
    #y_a are the boundary conditions 
    #tol is the tolerance for integrating y 
    
    #define our starting step 
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small 
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations 
    imax = 10000
    
    #set a iteration variable 
    i = 0
    
    #set the number of coupled odes to the 
    #size of y_a 
    nv = len(y_a)
    
    #set the initial conditions 
    x = np.full(1,a)
    y = np.ful((1,nv),y_a)
    
    #set a flag 
    flag = 1 
    
    #loop unitl we reach the right side 
    while(flag):
        
        #calculate y_i+1 
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step 
        h = h_new 
        
        #prevent an overshoot 
        if(xi+h_step>b):
            
            #take a smaller step 
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi, yi,nv,h,tol)
            
            #break
            flag = 0